## SQL

## 1. Очень усердные ученики.

Условие:

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. 
Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.
Задание: Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.
NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

Импортируем необходимые библиотеки:

In [1]:
import requests
import pandahouse as ph

Для решения задания создаем словарь connection с нужными параметрами (для подключения к default):

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

Создадим запрос к default.peas, позволяющий отфильтровать учеников, решивших более 20 задач за текущий месяц.

In [3]:
query_1 = '''
SELECT month_date,
       count(DISTINCT st_id) AS diligent_student
FROM (
      SELECT toStartOfMonth(timest) AS month_date,
             st_id
      FROM default.peas
      WHERE correct = 1
      GROUP BY month_date, st_id
      HAVING count(st_id)>=20
     )
GROUP BY month_date
   '''
diligent_student = ph.read_clickhouse(query=query_1, connection=connection_default)
diligent_student

,month_date,diligent_student
0,2021-10-01,136


В результате выполнения запроса получили - 136 учердных учеников, дата - 01-10-2021 (отличается от текущей, не смотря на то, что в запросе используется "toStartOfMonth()", делаем вывод, что последние данные в "default.peas"отображены за октябрь 2021)

## 2 Оптимизация воронки

Условие: 

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Задача: Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU;

ARPAU;

CR в покупку;

СR активного пользователя в покупку;

CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах. Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [4]:
query_2 =  '''
SELECT d.test_grp,
       sum_all_money/cnt_all AS ARPU,
       sum_active_money/cnt_active_all AS ARPAU,
       cnt_all_paid*100/cnt_all AS CR, 
       -- CR в покупку 
       cnt_all_active_paid*100/cnt_active_all AS CR_active, 
       -- СR активного пользователя в покупку 
       cnt_all_active_paid_math*100/cnt_active_math AS CR_active_math 
       -- CR пользователя из активности по математике в покупку курса по математике
FROM 
    (
    SELECT s.test_grp AS test_grp, 
           -- -- производим необходимые расчеты по данным из таблицы studs:
           COUNT(*) cnt_all, 
           -- для расчета ARPU, CR
           SUM(c.sum_money) AS sum_all_money,
           -- для расчета ARPU 
           COUNT(IF(c.check=1, 1, null)) AS cnt_all_paid,
           -- для расчета CR
           COUNT(IF(c.check=1, p.cnt_active, null)) AS cnt_all_active_paid,
           -- для расчета СR активного пользователя в покупку 
           COUNT(IF(c.check=1, p.cnt_active_math, null)) AS cnt_all_active_paid_math,
           -- для расчета CR пользователя из активности по математике в покупку курса по математике
           SUM(IF(p.cnt_active=1,c.sum_money,0)) AS sum_active_money,
           -- для расчета ARPAU
           SUM(c.cnt_math_paid) AS cnt_math_paid,
           SUM(c.math_paid_sum) AS math_paid_sum,
           SUM(p.cnt_active) AS cnt_active_all,
           -- для расчета ARPAU, CR_active
           SUM(p.cnt_active_math) AS cnt_active_math
           -- для расчета CR_active_math
    FROM studs s 
    LEFT JOIN -- добавляем информацию о покупках из таблицы final_project_check
            (
            SELECT c.st_id,
                   1 check, --  отражает факт покупки (1 - была покупка)
                   SUM(c.money) AS sum_money,
                   MAX(IF(c.subject = 'Math', 1, 0)) AS cnt_math_paid,
                   SUM(IF(c.subject = 'Math', c.money, 0)) AS math_paid_sum
                  
            FROM final_project_check c
            GROUP BY c.st_id
            ) c
    ON (c.st_id = s.st_id)
    LEFT JOIN  -- добавляем информацию об активных пользователях из таблицы peas
            (
            SELECT p.st_id,
                   1 check, -- отражает факт покупки (1 - была покупка)
                   IF(COUNT(*)>10,1,NULL) AS cnt_active,             
                   IF(COUNT(IF(p.subject = 'Math', 1, NULL)) >= 2, 1, NULL) AS cnt_active_math
            FROM peas p
            WHERE p.correct = 1
            GROUP BY p.st_id
            ) p 
    ON (p.st_id = s.st_id)
    GROUP BY s.test_grp
    ) d
'''
final_information = ph.read_clickhouse(query=query_2, connection=connection_default)
final_information

,test_grp,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.983607,10393.700787,4.918033,11.023622,10.204082
1,pilot,11508.474576,29739.583333,10.847458,26.041667,14.285714
